In [1]:
####get gedi training data as geopandas df

import geopandas as gpd
import pandas as pd
filename = "20230125T1201__L2A__subset_saldi_06(1).gpkg"
gdf = gpd.read_file(filename)

def classify_gedi(gdf):
    gdf_c = gdf.copy()
    gdf_c.loc[gdf["Relative Height bin98 (cm)"] <= 300, 'Relative Height bin98 (cm)'] = 300
    gdf_c.loc[(gdf["Relative Height bin98 (cm)"] > 300) & (gdf["Relative Height bin98 (cm)"] <= 600), 'Relative Height bin98 (cm)'] = 600
    gdf_c.loc[(gdf["Relative Height bin98 (cm)"] > 600) & (gdf["Relative Height bin98 (cm)"] <= 1000), 'Relative Height bin98 (cm)'] = 1000
    gdf_c.loc[gdf["Relative Height bin98 (cm)"] > 1000, 'Relative Height bin98 (cm)'] = 10000
    gdf = gdf_c.replace({'Relative Height bin98 (cm)' : {300 : '<3 m', 600 : '3 - 6 m', 1000 : '6 - 10 m', 10000 : '> 10 m'}})
    return gdf

gdf_recl = classify_gedi(gdf=gdf)

def leaf_off_subset(gdf):
    gdf['Acquisition Time'] = pd.to_datetime(gdf['Acquisition Time'])
    gdf = gdf.loc[(gdf["Acquisition Time"].dt.month >= 6) & (gdf["Acquisition Time"].dt.month <= 9), :]
    return gdf

gdf_recl_lo = leaf_off_subset(gdf=gdf_recl)
gdf_lo = leaf_off_subset(gdf=gdf)

In [187]:
from sdc.load import load_product

ds = load_product(product="s1_rtc", vec="geo419b_aoi.geojson", time_range=("2019-01-01", "2022-01-01"), apply_mask=False)
#ds.vv.values

In [188]:
ds.vv

<xarray.DataArray 'vv' (time: 258, latitude: 1336, longitude: 1854)>
dask.array<vv, shape=(258, 1336, 1854), dtype=float32, chunksize=(258, 510, 510), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 -25.0 -25.0 -25.0 ... -25.27 -25.27 -25.27
  * longitude    (longitude) float64 31.27 31.27 31.27 ... 31.64 31.64 31.64
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2019-01-06T03:18:46.924630 ... 2021-12...

In [210]:
# this results in bin centers on 100, 300, ...

ds_40 = ds.coarsen(longitude=2).mean().coarsen(latitude=2).mean()

In [211]:
#ds_40.isel(time=0).vv.plot()

In [212]:
ds_40.vv

<xarray.DataArray 'vv' (time: 258, latitude: 668, longitude: 927)>
dask.array<mean_agg-aggregate, shape=(258, 668, 927), dtype=float32, chunksize=(258, 255, 255), chunktype=numpy.ndarray>
Coordinates:
  * latitude     (latitude) float64 -25.0 -25.0 -25.01 ... -25.27 -25.27 -25.27
  * longitude    (longitude) float64 31.27 31.27 31.27 ... 31.64 31.64 31.64
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2019-01-06T03:18:46.924630 ... 2021-12...

In [195]:
#6-9
ds_clip = ds_40.sel(time=ds_40.time.dt.month.isin([6,7,8,9]))

In [196]:
ds_clip


<xarray.Dataset>
Dimensions:      (latitude: 1336, longitude: 1854, time: 86)
Coordinates:
  * latitude     (latitude) float64 -25.0 -25.0 -25.0 ... -25.27 -25.27 -25.27
  * longitude    (longitude) float64 31.27 31.27 31.27 ... 31.64 31.64 31.64
    spatial_ref  int32 4326
  * time         (time) datetime64[ns] 2019-06-03T16:30:37.017705 ... 2021-09...
Data variables:
    vv           (time, latitude, longitude) float32 dask.array<chunksize=(86, 510, 510), meta=np.ndarray>
    vh           (time, latitude, longitude) float32 dask.array<chunksize=(86, 510, 510), meta=np.ndarray>
    area         (time, latitude, longitude) float32 dask.array<chunksize=(86, 510, 510), meta=np.ndarray>

In [228]:
# Read the GeoJSON file as a GeoPandas DataFrame
clip_area = gpd.read_file('geo419b_aoi.geojson')

# Perform the clipping operation
gdf_recl_lo = gpd.clip(gdf_recl_lo, clip_area)


In [229]:
gdf_recl_lo

,Shot Number,Acquisition Time,Relative Height bin95 (cm),Relative Height bin98 (cm),geometry
170073,9.614110e+16,2020-08-23 12:53:42,250.0,3 - 6 m,POINT (31.50672 -25.26511)
170072,9.614110e+16,2020-08-23 12:53:42,220.0,<3 m,POINT (31.50635 -25.26472)
170071,9.614110e+16,2020-08-23 12:53:42,235.0,<3 m,POINT (31.50598 -25.26433)
170070,9.614110e+16,2020-08-23 12:53:42,228.0,<3 m,POINT (31.50561 -25.26394)
168451,9.614080e+16,2020-08-23 12:53:42,288.0,3 - 6 m,POINT (31.51990 -25.27076)
...,...,...,...,...,...
391437,4.504080e+16,2019-09-28 23:17:24,553.0,6 - 10 m,POINT (31.58433 -25.00421)
392156,4.504110e+16,2019-09-28 23:17:24,362.0,6 - 10 m,POINT (31.57775 -25.00555)
392155,4.504110e+16,2019-09-28 23:17:24,321.0,3 - 6 m,POINT (31.57738 -25.00515)
392154,4.504110e+16,2019-09-28 23:17:24,273.0,3 - 6 m,POINT (31.57701 -25.00476)


In [230]:
ds_month_min = ds_clip.resample(time='1M').min(dim='time')
ds_month_max = ds_clip.resample(time='1M').max(dim='time')
ds_month_mean = ds_clip.resample(time='1M').mean(dim='time')
ds_month_median = ds_clip.resample(time='1M').median(dim='time')
ds_month_std = ds_clip.resample(time='1M').std(dim='time')


ds_month_quantiles = ds_clip.resample(time='1M').quantile(dim='time', q=[0.05, 0.25, 0.5, 0.75, 0.95])


In [231]:
#from sdc.utils import ds_nanquantiles
#ds_month_quantiles = ds_nanquantiles(ds = ds_clip, quantiles=(0.05,0.25,0.75,0.95))




In [232]:
### berechne RVI
ds_clip_rvi = (4*ds_clip.vh)/(ds_clip.vv+ds_clip.vh)

In [233]:
#features for RVI
ds_month_rvi_min = ds_clip_rvi.resample(time='1M').min(dim='time')
ds_month_rvi_max = ds_clip_rvi.resample(time='1M').max(dim='time')
ds_month_rvi_mean = ds_clip_rvi.resample(time='1M').mean(dim='time')
ds_month_rvi_median = ds_clip_rvi.resample(time='1M').median(dim='time')
ds_month_rvi_std = ds_clip_rvi.resample(time='1M').std(dim='time')
ds_month_rvi_quantiles = ds_clip_rvi.resample(time='1M').quantile(dim='time', q=[0.05, 0.25, 0.5, 0.75, 0.95])

In [280]:
%%time
#list_min_vv = []
#list_min_vh = []
#list_pix = []
def get_pixel_lists(ds):
    liste = []
    for i in range(len(gdf_recl_lo)):
        ex = gdf_recl_lo["geometry"].iloc[i].coords[0]
        difflon = (float(ds.longitude[1]) - float(ds.longitude[0]))*0.5
        latmin = (ex[1]- (difflon))
        latmax = (ex[1]+ (difflon))
        lonmin = (ex[0]- (difflon))
        lonmax = (ex[0]+ (difflon))

        # Create conditions for latitude and longitude to extract one pixel
        lat_condition = ((ds.latitude >= (latmin)) & (ds.latitude <= (latmax)))
        lon_condition = ((ds.longitude >= (lonmin)) & (ds.longitude <= (lonmax)))

        # Use 'where' to filter the dataset based on both conditions
        pix = ds.where(lat_condition & lon_condition, drop=True)
        #get correct timestamp
        time = gdf_recl_lo["Acquisition Time"].iloc[i]
        liste.append(pix.sel(time=time, method='nearest'))#.compute()))
        liste[i] = float(liste[i])
    return liste

CPU times: user 152 µs, sys: 50 µs, total: 202 µs
Wall time: 74.9 µs


In [301]:
df = pd.DataFrame()
df['min_vv'] = []
df.to_csv('speichern.csv', index=False)

In [289]:
%%time
%px
df = pd.read_csv('speichern.csv')
ds_month_min_vv = ds_month_min.vv.compute()
df['min_vv'] = get_pixel_lists(ds=ds_month_min_vv)
df.to_csv('speichern.csv', index=False)

KeyboardInterrupt: 

In [45]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_max_vv = ds_month_max.vv.compute()
df['max_vv'] = get_pixel_lists(ds=ds_month_max_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 14s, sys: 36.3 s, total: 8min 51s
Wall time: 7min 16s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [46]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_mean_vv = ds_month_mean.vv.compute()
df['mean_vv'] = get_pixel_lists(ds=ds_month_mean_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 7min 50s, sys: 32.8 s, total: 8min 23s
Wall time: 6min 37s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [47]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_median_vv = ds_month_median.vv.compute()
df['median_vv'] = get_pixel_lists(ds=ds_month_median_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 2s, sys: 34.5 s, total: 8min 36s
Wall time: 6min 40s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [48]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_std_vv = ds_month_std.vv.compute()
df['std_vv'] = get_pixel_lists(ds=ds_month_std_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 2s, sys: 37.6 s, total: 8min 39s
Wall time: 7min 16s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [49]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_05_vv = ds_month_quantiles.vv.sel(quantile=0.05).compute()
df['05_vv'] = get_pixel_lists(ds=ds_month_05_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 10min 35s, sys: 50.3 s, total: 11min 25s
Wall time: 47min 25s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [50]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_25_vv = ds_month_quantiles.vv.sel(quantile=0.25).compute()
df['25_vv'] = get_pixel_lists(ds=ds_month_25_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 47s, sys: 54.8 s, total: 10min 42s
Wall time: 48min 32s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [68]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_75_vv = ds_month_quantiles.vv.sel(quantile=0.75).compute()
df['75_vv'] = get_pixel_lists(ds=ds_month_75_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 41s, sys: 45.9 s, total: 9min 27s
Wall time: 46min 58s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [69]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_95_vv = ds_month_quantiles.vv.sel(quantile=0.95).compute()
df['95_vv'] = get_pixel_lists(ds=ds_month_95_vv)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 52s, sys: 56 s, total: 10min 48s
Wall time: 50min 9s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [70]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_min_vh = ds_month_min.vh.compute()
df['min_vh'] = get_pixel_lists(ds=ds_month_min_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 28s, sys: 40.8 s, total: 9min 9s
Wall time: 6min 50s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [71]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_max_vh = ds_month_max.vh.compute()
df['max_vh'] = get_pixel_lists(ds=ds_month_max_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 32s, sys: 41.1 s, total: 9min 13s
Wall time: 8min 19s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [72]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_mean_vh = ds_month_mean.vh.compute()
df['mean_vh'] = get_pixel_lists(ds=ds_month_mean_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 12s, sys: 40 s, total: 8min 52s
Wall time: 6min 48s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [73]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_median_vh = ds_month_median.vh.compute()
df['median_vh'] = get_pixel_lists(ds=ds_month_median_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 13s, sys: 40 s, total: 8min 53s
Wall time: 6min 50s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [74]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_std_vh = ds_month_std.vh.compute()
df['std_vh'] = get_pixel_lists(ds=ds_month_std_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 23s, sys: 41.4 s, total: 9min 4s
Wall time: 6min 51s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [75]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_05_vh = ds_month_quantiles.vh.sel(quantile=0.05).compute()
df['05_vh'] = get_pixel_lists(ds=ds_month_05_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 44s, sys: 56.1 s, total: 10min 40s
Wall time: 46min 1s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [32]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_25_vh = ds_month_quantiles.vh.sel(quantile=0.25).compute()
df['25_vh'] = get_pixel_lists(ds=ds_month_25_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 11min 47s, sys: 52.9 s, total: 12min 40s
Wall time: 46min 37s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [27]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_75_vh = ds_month_quantiles.vh.sel(quantile=0.75).compute()
df['75_vh'] = get_pixel_lists(ds=ds_month_75_vh)
df.to_csv('speichern.csv', index=False)

KilledWorker: Attempted to run task ('broadcast_to-concatenate-transpose-wrapper-wrapper_0-getitem-b216eccdd21350f25a3f4302f6704b15', 11, 0, 1, 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://172.18.1.9:38879. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [34]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_95_vh = ds_month_quantiles.vh.sel(quantile=0.95).compute()
df['95_vh'] = get_pixel_lists(ds=ds_month_95_vh)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 29s, sys: 54.7 s, total: 10min 24s
Wall time: 47min 27s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [35]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_min = ds_month_rvi_min.compute()
df['min_rvi'] = get_pixel_lists(ds=ds_month_rvi_min)
df.to_csv('speichern.csv', index=False)

CPU times: user 7min 43s, sys: 33.9 s, total: 8min 17s
Wall time: 6min 44s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [36]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_max = ds_month_rvi_max.compute()
df['max_rvi'] = get_pixel_lists(ds=ds_month_rvi_max)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 12s, sys: 36.4 s, total: 8min 49s
Wall time: 7min 4s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [37]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_mean = ds_month_rvi_mean.compute()
df['mean_rvi'] = get_pixel_lists(ds=ds_month_rvi_mean)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 7s, sys: 37.8 s, total: 8min 45s
Wall time: 6min 50s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [38]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_median = ds_month_rvi_median.compute()
df['median_rvi'] = get_pixel_lists(ds=ds_month_rvi_median)
df.to_csv('speichern.csv', index=False)

CPU times: user 8min 16s, sys: 35 s, total: 8min 51s
Wall time: 7min 4s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [39]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_std = ds_month_rvi_std.compute()
df['std_rvi'] = get_pixel_lists(ds=ds_month_rvi_std)
df.to_csv('speichern.csv', index=False)

CPU times: user 7min 18s, sys: 28.5 s, total: 7min 46s
Wall time: 6min 23s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [40]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_05 = ds_month_rvi_quantiles.sel(quantile=0.05).compute()
df['05_rvi'] = get_pixel_lists(ds=ds_month_rvi_05)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 41s, sys: 56.6 s, total: 10min 38s
Wall time: 50min 16s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [49]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_25 = ds_month_rvi_quantiles.sel(quantile=0.25).compute()
df['25_rvi'] = get_pixel_lists(ds=ds_month_rvi_25)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 38s, sys: 54.8 s, total: 10min 33s
Wall time: 48min 3s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [42]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_75 = ds_month_rvi_quantiles.sel(quantile=0.75).compute()
df['75_rvi'] = get_pixel_lists(ds=ds_month_rvi_75)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 16s, sys: 48 s, total: 10min 4s
Wall time: 46min 53s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [48]:
%%time
df = pd.read_csv('speichern.csv')
ds_month_rvi_95 = ds_month_rvi_quantiles.sel(quantile=0.95).compute()
df['95_rvi'] = get_pixel_lists(ds=ds_month_rvi_95)
df.to_csv('speichern.csv', index=False)

CPU times: user 9min 52s, sys: 1min 1s, total: 10min 54s
Wall time: 52min 53s


/home/ga59wig/micromamba/envs/geo419b_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [44]:
######               27 features

In [18]:
#gdf_recl_lo.to_file('gdf_recl_lo_res.gpkg', driver='GPKG')